In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
import pyinaturalist as iNat
roseObs = pd.read_csv("processedData/roseObs")
labeledRoseObs = roseObs.dropna()
labeledRoseObs["isFlowering"] = labeledRoseObs['reproductiveCondition'].isin(['flowering','flowering|fruiting','flowering|fruiting|flower budding','flowering|flower budding']).astype(int)
numList = []

for link in labeledRoseObs["references"]:
    numList.append(int(link.split("/")[-1]))

labeledRoseObs["observationNumber"] = numList

Maybe try to use log10 in your analysis. We can also compute the correlation. We could compare mean and median of variables as a function of categorical variables.Even better we can make a box and whisker plot.


In [ ]:
kfold = KFold(n_splits=5, random_state=44, shuffle=True)

accuracy = np.zeros((4, 5))
## sets a split counter
i = 0

## loop through the kfold here
for train_index, test_index in kfold.split(labeledRoseObs):
    ## get the cv training set
    roses_tt = labeledRoseObs.iloc[train_index]
    
    ## get the cv holdout set
    roses_ho = labeledRoseObs.iloc[test_index]
    
    ## "Fit" and get ho mse for the baseline model
    baseline_pred = []
    for month in roses_ho["month"]:
        # a better way to do this would be to check which months have a majority of observations in flower
        if month in [5,6,7]:
            baseline_pred.append(1)
        else:
            baseline_pred.append(0)
    
    accuracy[0, i] = 1 - sum(abs(np.array(roses_ho["isFlowering"]) - np.array(baseline_pred)))/len(roses_ho)
    
    ## Fit and get the mses for the remaining models
    LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(roses_tt[["year", "month", "day", "decimalLatitude", "decimalLongitude"]], roses_tt["isFlowering"])
    pred = LR.predict(roses_ho[["year", "month", "day", "decimalLatitude", "decimalLongitude"]])
    accuracy[1,i] = 1 - sum(abs(np.array(roses_ho["isFlowering"]) - np.array(pred)))/len(roses_ho)
    
    RF = RandomForestClassifier(n_estimators= 200, max_depth=400, random_state=0)
    RF.fit(roses_tt[["year", "month", "day", "decimalLatitude", "decimalLongitude"]], roses_tt["isFlowering"])
    pred = RF.predict(roses_ho[["year", "month", "day", "decimalLatitude", "decimalLongitude"]])
    accuracy[2,i] = 1 - sum(abs(np.array(roses_ho["isFlowering"]) - np.array(pred)))/len(roses_ho)

    GB = GradientBoostingClassifier(n_estimators = 400, max_leaf_nodes = 10, max_depth =  None, random_state = 2, min_samples_split = 10)
    GB.fit(roses_tt[["year", "month", "day", "decimalLatitude", "decimalLongitude"]], roses_tt["isFlowering"])
    pred = GB.predict(roses_ho[["year", "month", "day", "decimalLatitude", "decimalLongitude"]])
    accuracy[3,i] = 1 - sum(abs(np.array(roses_ho["isFlowering"]) - np.array(pred)))/len(roses_ho)

    
    ## Increasing the counter
    i = i + 1

accuracy

In [ ]:
np.mean(accuracy,axis = 1)